In [1]:
import os
import re

import django
import fs.path
import numpy as np
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist
from fs.osfs import OSFS

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "multidex.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from plotter.models import *
from plotter_utils import modeldict

In [2]:
for spec in ZSpec.objects.all():
    spec.delete()

In [3]:
input_fs = OSFS("/home/michael/Desktop/mcam_spect_data_conversion/data/zcam")
output_fs = OSFS('.')
output_image_dir = output_fs.getsyspath("static_in_pro/our_static/img")

In [4]:
def reg_seq_id(seq_id_str):
    seq_id_str = seq_id_str.replace(" ", "")
    return 'zcam' + format(int(seq_id_str.split('zcam')[1]), "0>5d")

metaframe = pd.read_csv(input_fs.getsyspath('Metadata-marslab.csv'))
metaframe.columns = [column.lower() for column in metaframe.columns]
# add NaNs back in so we can programmatically delete them
metaframe.replace('-', np.nan, inplace=True)
# we're turning these to ints when we ingest them,
# but python doesn't like statements like int('3.0'),
# so turn to float as an intermediate step
for column in [
    'sol', 'site', 'drive', 'rover_elevation', 'target_elevation', 'tau',
    'focal_distance', 'incidence_angle', 'emission_angle',
    'phase_angle', 'l_s', 'site', 'drive', 'lat', 'lon', 'odometry',
]:
    metaframe[column] = metaframe[column].astype('float')
metaframe['ltst'] = metaframe['ltst'].astype('datetime64')
metaframe['seq_id'] = metaframe['seq_id'].map(reg_seq_id)
metaframe.loc[
    metaframe['sclk'].notna(), 'sclk'
] = metaframe.loc[metaframe['sclk'].notna(), 'sclk'].astype(float).astype(int)

In [5]:
temp_images = [
    image for image in input_fs.listdir('temp_images')
]

In [7]:
spec_files = [
    file for file in input_fs.listdir('') if (
        file.endswith('-marslab.csv') and file.startswith('Sol'))
]
for spec_file in spec_files:
    frame = pd.read_csv(input_fs.getsyspath(spec_file))
    frame.columns = [column.lower() for column in frame.columns]
    observation = metaframe.loc[metaframe['seq_id'] == frame['seq_id'][0]]
    # hacky hacky -- but temporary
    if len(observation) > 1: 
        pointing = re.search(r"(?<=Pointing)\d", spec_file).group()
        observation = observation.loc[observation['name'].str.contains(str(pointing))]
    assert len(observation == 1)
    for _, row in frame.iterrows():
        if row['float'] == 'Y':
            row['float'] = True
        else:
            row['float'] = False
        row = row.replace(['-','',' '], np.nan).dropna()
        # if there are missing filters anywhere in the column, including for other
        # spectra, pandas will read the column
        # as object / string, which will cause confusion when we
        # compute averaged filters
        for filt in ZSpec.filters:
            if filt in row.index:
                row[filt] = float(row[filt])
        # because we're manually associating
        row.drop(['sol', 'seq_id', 'instrument'], inplace=True)
        metadata = dict(row) | observation.iloc[0].dropna().to_dict() | {
            'filename': spec_file
        }
        spectrum = ZSpec(**metadata)
        spectrum.clean()
        spectrum.save()

In [8]:
len(ZSpec.objects.all())

111

In [10]:
output_image_dir

'/home/michael/Desktop/multidex_flat_branch/multidex/static_in_pro/our_static/img'

In [13]:
spec.seq_id

'zcam03106'

In [14]:
os.listdir(output_image_dir)

['roi_full', 'roi_browse']

In [18]:
spec.filename

'Sol33_zcam03106_Ahyeeh_Pointing2-marslab.csv'

In [19]:
import re

In [24]:
for spec in ZSpec.objects.all():
    images = [
        image for image in os.listdir(output_image_dir + '/roi_browse')
        if spec.seq_id in image
    ]
    if 'pointing' in spec.filename.lower():
        pointing = re.search(r"(?<=Pointing)\d", spec_file).group()
        images = [image for image in images if 'pointing' + pointing in image]
    image_dict = {}
    for image in images:
        if 'rgb' in image:
            image_dict['rgb'] = image
        if 'enhanced' in image:
            image_dict['enhanced'] = image
    spec.images = str(image_dict)
    spec.clean()
    spec.save()

In [3]:
from random import choice

In [4]:
spec = choice(ZSpec.objects.all())

In [21]:
from plotter.spectrum_ops import *

In [15]:
unc = """L0R:3.3
L0G:3.3
L0B:3.7
L1:1.4
L2:1.1
L3:0.2
L4:1.8
L5:1.6
L6:0.4
R0R:3.7
R0G:4.1
R0B:4.6
R1:0.4
R2:0.3
R3:0.6
R4:0.5
R5:0.8
R6:0.4""".split("\n")

In [19]:
unc

['L0R:3.3',
 'L0G:3.3',
 'L0B:3.7',
 'L1:1.4',
 'L2:1.1',
 'L3:0.2',
 'L4:1.8',
 'L5:1.6',
 'L6:0.4',
 'R0R:3.7',
 'R0G:4.1',
 'R0B:4.6',
 'R1:0.4',
 'R2:0.3',
 'R3:0.6',
 'R4:0.5',
 'R5:0.8',
 'R6:0.4']

In [22]:
{
    pair.split(":")[0]: float(pair.split(":")[1])
    for pair in unc
}

{'L0R': 3.3,
 'L0G': 3.3,
 'L0B': 3.7,
 'L1': 1.4,
 'L2': 1.1,
 'L3': 0.2,
 'L4': 1.8,
 'L5': 1.6,
 'L6': 0.4,
 'R0R': 3.7,
 'R0G': 4.1,
 'R0B': 4.6,
 'R1': 0.4,
 'R2': 0.3,
 'R3': 0.6,
 'R4': 0.5,
 'R5': 0.8,
 'R6': 0.4}

In [22]:
filter_df = filter_df_from_queryset(
    ZSpec.objects.all(),
    average_filters=False,
    scale_to=None,
    r_star=True,
    )

In [29]:
from marslab.compatibility import INSTRUMENT_UNCERTAINTIES
from itertools import chain, product

In [129]:
spec_model = ZSpec
instrument = spec_model.instrument
unc = INSTRUMENT_UNCERTAINTIES[instrument]
function = ratio
# filters = ['L5', 'R3', 'L4']
filters = ['L5', 'R3']

In [ ]:
def compute_minmax_spec_error(filter_df, spec_model, spec_op, *filters):
    # cartesian product of these sets gives all possible sign combos for error high, error low,
    # i.e., crude bounds for the hull of the range of possible measurements
    unc = INSTRUMENT_UNCERTAINTIES[spec_model.instrument]
    corners = product(*[[1, -1] for filt in filters])
    bounds_df_list = []
    # apply these signs to uncertainty values, getting a list of dataframes
    # giving values of all measurements in set at the upper / lower bound combinations
    # for uncertainties associated with each relevant filter.
    for corner in corners:
        corner_series_list = []
        for filt_ix, sign in enumerate(corner):
            filt = filters[filt_ix]
            corner_series_list.append(
                filter_df[filt]  + filter_df[filt] * corner[filt_ix] * unc[filt] / 100
            )
        corner_df = pd.concat(corner_series_list, axis=1)
        # record the value of the spectrum op for each of these bounding dataframes.
        bounds_df_list.append(function(corner_df, spec_model, *filters)[0])   
    # compute the nominal value and compare it to values at these bounds
    possible_values = pd.concat(bounds_df_list, axis=1)
    nominal_value = function(filter_df, spec_model, *filters)[0]
    offsets = possible_values.sub(nominal_value, axis=0) 
    # then min / max of each of these gives us an error estimate for each spectrum
    return offsets.min(axis=1), offsets.max(axis=1)

In [165]:
# cartesian product of these sets gives all possible sign combos for error high, error low,
# i.e., crude bounds for the hull of the range of possible measurements
corners = product(*[[1, -1] for filt in filters])
bounds_df_list = []
# apply these signs to uncertainty values, getting a list of dataframes
# giving values of all measurements in set at the upper / lower bound combinations
# for uncertainties associated with each relevant filter.
for corner in corners:
    corner_series_list = []
    for filt_ix, sign in enumerate(corner):
        filt = filters[filt_ix]
        corner_series_list.append(
            filter_df[filt]  + filter_df[filt] * corner[filt_ix] * unc[filt] / 100
        )
    corner_df = pd.concat(corner_series_list, axis=1)
    # record the value of the spectrum op for each of these bounding dataframes.
    bounds_df_list.append(function(corner_df, spec_model, *filters)[0])    

In [179]:
# compute the nominal value and compare it to values at these bounds
possible_values = pd.concat(bounds_df_list, axis=1)
nominal_value = function(filter_df, spec_model, *filters)[0]
offsets = possible_values.sub(nominal_value, axis=0) 
# then min / max of each of these gives us an error estimate for each spectrum
error_low = offsets.min(axis=1)
error_high = offsets.max(axis=1)

In [108]:
nominal_value = function(filter_df, spec_model, *filters)[0]

In [114]:
possible_values

,0,1,2,3,4,5,6,7
1,-0.227605,0.002936,-0.246920,-0.012752,-0.467757,-0.192116,-0.495453,-0.214611
2,-0.272037,-0.015633,-0.294336,-0.033437,-0.555008,-0.241566,-0.588462,-0.268276
3,-0.334284,-0.113748,-0.354353,-0.130499,-0.581876,-0.320417,-0.610163,-0.344028
4,-0.226284,-0.011445,-0.244244,-0.026259,-0.446898,-0.193409,-0.471969,-0.214087
5,-0.361472,-0.147387,-0.381342,-0.164133,-0.605394,-0.352954,-0.633094,-0.376298
...,...,...,...,...,...,...,...,...
107,-0.418209,-0.187692,-0.440520,-0.206377,-0.695613,-0.420007,-0.727604,-0.446799
108,-0.338147,-0.192867,-0.351338,-0.204626,-0.491981,-0.330000,-0.508397,-0.344634
109,-0.412618,-0.237721,-0.429415,-0.252439,-0.612810,-0.413128,-0.634743,-0.432345
110,-0.288937,-0.063716,-0.308742,-0.080060,-0.534162,-0.266092,-0.562302,-0.289315


In [113]:
nominal_value

1     -0.221747
2     -0.270365
3     -0.338700
4     -0.220741
5     -0.368213
         ...   
107   -0.431098
108   -0.341197
109   -0.421762
110   -0.289108
111   -0.428285
Length: 111, dtype: float64

In [104]:
nominal_value = function(filter_df, spec_model, *filters)[0]
bound = function(bounds_df_list[0], spec_model, *filters)[0] 
bound - nominal_value

1     -0.005858
2     -0.001673
3      0.004415
4     -0.005543
5      0.006741
         ...   
107    0.012889
108    0.003050
109    0.009145
110    0.000171
111    0.009755
Length: 111, dtype: float64

1      0.195836
2      0.193705
3      0.255388
4      0.209990
5      0.269752
         ...   
107    0.273707
108    0.386815
109    0.365840
110    0.228434
111    0.355110
Length: 111, dtype: float64

1     -inf
2     -inf
3     -inf
4     -inf
5     -inf
      ... 
107   -inf
108   -inf
109   -inf
110   -inf
111   -inf
Length: 111, dtype: float64

In [86]:
# note that band_depth_min is a special case

nominal_value = band_avg(filter_df, ZSpec, 'L1', 'R1')[0]
band_avg(bounds_df_list[0], ZSpec, 'L1', 'R1')[0] - nominal_value

1      0.009
2      0.009
3      0.009
4      0.009
5      0.009
       ...  
107    0.009
108    0.009
109    0.009
110    0.009
111    0.009
Length: 111, dtype: float64

In [82]:
corner_df - nominal_value

1     -0.009
2     -0.009
3     -0.009
4     -0.009
5     -0.009
       ...  
107   -0.009
108   -0.009
109   -0.009
110   -0.009
111   -0.009
Length: 111, dtype: float64

In [45]:
list(product(filter_df_low.columns, filter_df_high))

AttributeError: 'Series' object has no attribute 'columns'

In [35]:
filter_df_low

1      0.193094
2      0.190993
3      0.251812
4      0.207051
5      0.265976
         ...   
107    0.272612
108    0.385268
109    0.364377
110    0.227521
111    0.353689
Length: 222, dtype: float64